In [27]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [28]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [29]:
# load data:
df_ht = pd.read_csv("hotel_bookings.csv")

In [30]:
# Replace missing values:
# agent: If no agency is given, booking was most likely made without one.
# company: If none given, it was most likely private.
# rest schould be self-explanatory.
nan_replacements = {"children:": 0.0,"country": "Unknown", "agent": 0, "company": 0}
df_clean = df_ht.fillna(nan_replacements)

# "meal" contains values "Undefined", which is equal to SC.
df_clean["meal"].replace("Undefined", "SC", inplace=True)

# Some rows contain entreis with 0 adults, 0 children and 0 babies. 
# I'm dropping these entries with no guests.
zero_guests = list(df_clean.loc[df_clean["adults"]
                   + df_clean["children"]
                   + df_clean["babies"]==0].index)
df_clean.drop(df_clean.index[zero_guests], inplace=True)

In [31]:
portugal_df = df_clean[df_clean["country"] == 'PRT']
france_df = df_clean[df_clean["country"] == 'FRA']
england_df = df_clean[df_clean["country"] == 'GBR']
germany_df = df_clean[df_clean["country"] == 'DEU']
italy_df = df_clean[df_clean["country"] == 'ITA']

portugal_df.head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.00,0,0,Check-Out,2015-07-01
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,...,No Deposit,0.0,0.0,0,Transient,107.00,0,0,Check-Out,2015-07-03
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,...,No Deposit,303.0,0.0,0,Transient,103.00,0,1,Check-Out,2015-07-03
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,...,No Deposit,240.0,0.0,0,Transient,82.00,0,1,Canceled,2015-05-06
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,...,No Deposit,15.0,0.0,0,Transient,105.50,0,0,Canceled,2015-04-22
10,Resort Hotel,1,23,2015,July,27,1,0,4,2,...,No Deposit,240.0,0.0,0,Transient,123.00,0,0,Canceled,2015-06-23
11,Resort Hotel,0,35,2015,July,27,1,0,4,2,...,No Deposit,240.0,0.0,0,Transient,145.00,0,0,Check-Out,2015-07-05
14,Resort Hotel,0,37,2015,July,27,1,0,4,2,...,No Deposit,241.0,0.0,0,Transient,94.71,0,0,Check-Out,2015-07-05
16,Resort Hotel,0,37,2015,July,27,1,0,4,2,...,No Deposit,8.0,0.0,0,Contract,97.50,0,0,Check-Out,2015-07-05


In [53]:
clean_portugal_df = portugal_df[['stays_in_weekend_nights',
       'stays_in_week_nights',  'children', 'babies', 'previous_cancellations',
       'previous_bookings_not_canceled',  'booking_changes', 
       'required_car_parking_spaces', 'total_of_special_requests']]
#portugal_df.dropna(axis=0, subset =)
clean_portugal_df.head(10)

,stays_in_weekend_nights,stays_in_week_nights,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,required_car_parking_spaces,total_of_special_requests
0,0,0,0.0,0,0,0,3,0,0
1,0,0,0.0,0,0,0,4,0,0
6,0,2,0.0,0,0,0,0,0,0
7,0,2,0.0,0,0,0,0,0,1
8,0,3,0.0,0,0,0,0,0,1
9,0,3,0.0,0,0,0,0,0,0
10,0,4,0.0,0,0,0,0,0,0
11,0,4,0.0,0,0,0,0,0,0
14,0,4,0.0,0,0,0,0,0,0
16,0,4,0.0,0,0,0,0,0,0


In [42]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

# Encoding the datasets
clean_portugal_df_encoded = clean_portugal_df.applymap(hot_encode)
clean_portugal_df = clean_portugal_df_encoded

In [51]:
# Building the model
frq_items = apriori(clean_portugal_df, min_support = 0.05, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
print(rules.head())

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value nan